<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Data Crunching</div>

0. [Modules](#sec0) 
1. [Data Loading](#sec1)
2. [Extraction of Data](#sec2)
3. [Additional work (exploration)](#sec3)

# 0. <a id="sec0"></a> Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import math

# 1. <a id="sec1"></a> Data Loading & Cleansing

In [2]:
#help(pd.read_csv)

In [3]:
df_ticket_data = pd.read_csv("data/ticket_data.csv")
df_stations = pd.read_csv("data/stations.csv")
df_providers = pd.read_csv("data/providers.csv")
df_cities = pd.read_csv("data/cities.csv")

In [4]:
df_ticket_data.head()

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00+00,2017-10-13 20:10:00+00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542


In [5]:
df_cities.head()

,id,local_name,unique_name,latitude,longitude,population
0,5159,"Padua, Veneto, Italia",padua,45.406435,11.876761,209678.0
1,76,"Barcelona, Cataluña, España",barcelona,41.385064,2.173404,1611822.0
2,81,"Basel, Basel-Stadt, Schweiz",basel,47.593437,7.619812,NaN
3,259,"Erlangen, Bayern, Deutschland",erlangen,49.589674,11.011961,105412.0
4,11979,"Balș, Olt, România",balș,44.353354,24.095672,NaN


In [6]:
df_stations.head()

,id,unique_name,latitude,longitude
0,1,Aalen (Stuttgarter Straße),48.835296,10.092956
1,2,Aéroport Bordeaux-Mérignac,44.830226,-0.700883
2,3,Aéroport CDG,49.009900,2.559310
3,4,Aéroport de Berlin-Schönefeld,52.389446,13.520345
4,5,Aéroport de Dresden,51.123604,13.764737


In [7]:
df_providers.dropna()

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
8,17,9,31,expressbusMunchen,Expressbus Munchen,False,False,False,False,bus
9,18,9,33,berlinLinienBus,Berlin Linien Bus,False,False,False,False,bus
10,19,9,37,ostfrieslandexpress,OstfrieslandExpress,False,False,False,False,bus
11,20,9,41,icBus,IC Bus,False,False,False,False,bus
12,21,9,45,busandfly,Busandfly,False,False,False,False,bus
...,...,...,...,...,...,...,...,...,...,...
218,8380,10,5,train,TER,False,False,True,False,train
219,8381,10,6,coraillunea,Intercités de Nuit,False,False,True,False,train
220,8382,10,8,eurostar,Eurostar,False,False,True,False,train
221,8383,10,9,corail,Intercités,False,False,True,False,train


The dataset `ticket_data.csv` contains all the history of tickets proposed by Tictatrip.
<br>The dataset `cities.csv` enumerates for each trip the destination city and its position.
<br>The dataset `stations.csv` enumerates all the stations where there are stops.
<br>The dataset `providers.csv` provides information about the different providers referenced on the trips proposed by Tictatrip.

## Data Cleansing & Expanding

Before going further, it is important to identify some issues in the datasets and to expand it by adding useful columns for the analysis.

There are some NaN values in the column "o_station" and "d_station" in the dataframe `ticket_data.csv`. For this reason, the *ids* are float numbers instead of integers.

Let us express the times in Datetime type.
<br>The initial values of `departure_ts` and other dates are of types str and have the following format: 'YYYY-MM-DD hh:mm:ss+00'
<br>And we want the following string format: 
<br>'YYYY-MM-DD hh:mm:ss'
<br>The second will be enough in terms of duration precision!

In [8]:
def correct_string_date_format(date_string):
    return date_string[:19]
date_string = '2017-10-13 13:27:00+00'
correct_string_date_format(date_string)

'2017-10-13 13:27:00'

In [9]:
## we apply this function to any columns representing a time in our datasets

df_ticket_data["departure_datetime"] = pd.to_datetime(df_ticket_data['departure_ts'].apply(correct_string_date_format))
df_ticket_data["arrival_datetime"] = pd.to_datetime(df_ticket_data['arrival_ts'].apply(correct_string_date_format))
df_ticket_data["duration_datetime"] = df_ticket_data["arrival_datetime"] - df_ticket_data["departure_datetime"]
df_ticket_data["search_datetime"] = pd.to_datetime(df_ticket_data['search_ts'].apply(correct_string_date_format))

# Eventually since these 3 columns are not used as keys to join 
# other tables/datasets, we remove them:
df_ticket_data = df_ticket_data.drop(columns=['departure_ts', 'arrival_ts', 'search_ts'])


In [10]:
# Add a column for prices in €
df_ticket_data["price_in_euros"] = df_ticket_data["price_in_cents"]/100

In [11]:
df_ticket_data.head()

,id,company,o_station,d_station,price_in_cents,middle_stations,other_companies,o_city,d_city,departure_datetime,arrival_datetime,duration_datetime,search_datetime,price_in_euros
0,6795025,8385,NaN,NaN,4550,NaN,NaN,611,542,2017-10-13 14:00:00,2017-10-13 20:10:00,0 days 06:10:00,2017-10-01 00:13:31,45.5
1,6795026,9,63.0,1044.0,1450,"{149,418}",{13},611,542,2017-10-13 13:05:00,2017-10-14 06:55:00,0 days 17:50:00,2017-10-01 00:13:35,14.5
2,6795027,8377,5905.0,6495.0,7400,"{798,798,6794,6246}","{8377,8376}",611,542,2017-10-13 13:27:00,2017-10-14 21:24:00,1 days 07:57:00,2017-10-01 00:13:40,74.0
3,6795028,8377,5905.0,6495.0,13500,"{798,798,6794,6246}","{8377,8376}",611,542,2017-10-13 13:27:00,2017-10-14 11:02:00,0 days 21:35:00,2017-10-01 00:13:40,135.0
4,6795029,8381,5905.0,6495.0,7710,"{5983,5983}",{8380},611,542,2017-10-13 21:46:00,2017-10-14 19:32:00,0 days 21:46:00,2017-10-01 00:13:40,77.1


In [12]:
df_ticket_data.dtypes

id                              int64
company                         int64
o_station                     float64
d_station                     float64
price_in_cents                  int64
middle_stations                object
other_companies                object
o_city                          int64
d_city                          int64
departure_datetime     datetime64[ns]
arrival_datetime       datetime64[ns]
duration_datetime     timedelta64[ns]
search_datetime        datetime64[ns]
price_in_euros                float64
dtype: object

# 2. <a id="sec2"></a> Extraction of Data

<div class="alert alert-warning">
    
**Exercice: Extraction of relevant information** <br>
- Minimimum, maximum and average prices & duration per trip
- Difference of the average price & duration according to the means of transport (train, bus, car pooling) and according to the distance (Segmentation proposed: 0-200km, 201-800km, 800-2000km, 2000+km)
</div>

### Minimimum, maximum and average prices & duration per trip

This requires to use a `groupby` method to compute the aggregation functions such as `min`, `max` and `avg` on each trip.
<br>A trip is determined by the origin city(`o_city`) and the destination city (`d_city`).
<br>Now let us compute these values thanks to the dataframe we just rearranged.

In [13]:
def my_agg(x):
    names = {
        'Price mean': x['price_in_euros'].mean(),
        'Price std':  x['price_in_euros'].std(),
        'Price min':  x['price_in_euros'].min(),
        'Price max':  x['price_in_euros'].std(),
        'Duration mean': x['duration_datetime'].mean(),
        #'Duration std':  x['duration_datetime'].std(),
        'Duration min':  x['duration_datetime'].min(),
        'Duration max':  x['duration_datetime'].std()
    }

    return pd.Series(names, index=['Price mean', 'Price std', 'Price min', 'Price max', 
                                   'Duration mean', 'Duration min', 'Duration max'])

df_ticket_data.groupby(["o_city", "d_city"]).apply(my_agg)

Price mean  Price std  Price min  Price max   Duration mean  \
o_city d_city                                                                
5      23      203.200000  13.700365      186.0  13.700365        10:18:48   
6      227     117.550000  26.799347       98.6  26.799347        13:42:30   
       504      40.426667  20.763013       20.0  20.763013        08:17:24   
       628      27.975000   2.781337       26.0   2.781337        12:10:00   
       845       8.646269   1.773668        7.0   1.773668 01:19:54.626865   
...                   ...        ...        ...        ...             ...   
11938  126      42.048611   5.136722       30.0   5.136722 07:25:16.666666   
12124  1064     69.500000  13.435029       60.0  13.435029        17:10:00   
12166  857      53.000000        NaN       53.0        NaN        21:55:00   
12190  639       6.888889   0.993443        6.0   0.993443 01:28:53.333333   
       8937     98.000000        NaN       98.0        NaN        09:10:00   

              Duration min    Duration max  
o_city d_city                               
5      23         08:53:00 03:07:23.350034  
6      227        12:24:00 01:51:00.945878  
       504        05:36:00 01:57:12.175118  
       628        09:40:00 01:58:48.814768  
       845        01:00:00 00:21:32.999220  
...                    ...             ...  
11938  126        05:30:00 05:10:44.305855  
12124  1064       11:10:00 08:29:07.012947  
12166  857        21:55:00             NaT  
12190  639        01:10:00 00:28:28.112132  
       8937       09:10:00             NaT  

[1437 rows x 7 columns]

We obtain a dataframe with 1437 rows (which denotes the number of trips that results from the `groupby`).
To get an overview and a more readable and synthetic dataframe, we will compute the same metrics but over all the 71148 tickets registered in the history.

In [14]:
df_ticket_data[["price_in_euros", "duration_datetime"]].dropna().describe()

,price_in_euros,duration_datetime
count,74168.000000,74168
mean,43.827111,0 days 07:04:37.247600
std,37.393254,0 days 09:54:58.881351
min,3.000000,0 days 00:20:00
25%,19.000000,0 days 03:00:00
50%,33.500000,0 days 04:50:00
75%,52.500000,0 days 08:00:00
max,385.500000,20 days 12:51:00


### Difference of the average price & duration according to the means of transport (train, bus, car pooling) and according to the distance (Segmentation proposed: 0-200km, 201-800km, 800-2000km, 2000+km)

First, we gather the useful information in the table `df_ticket_data`.
<br>To do so, we will perform a left join operation and extract information from the datasets: 
- `df_cities`
- `df_stations`

In [15]:
# Beware: execute the cell just once
df_ticket_data = pd.merge(left=df_ticket_data, right=df_cities[['id', 'latitude', 'longitude']], how='left', left_on='o_city', right_on='id', suffixes=("", "_city_o"))
df_ticket_data = df_ticket_data.rename(columns={"latitude": "o_lat", "longitude": "o_long"})   
df_ticket_data = df_ticket_data.drop(columns=['id_city_o'])
df_ticket_data = pd.merge(left=df_ticket_data, right=df_cities[['id', 'latitude', 'longitude']], how='left', left_on='d_city', right_on='id', suffixes=("", "_city_d"))
df_ticket_data = df_ticket_data.rename(columns={"latitude": "d_lat", "longitude": "d_long"})
df_ticket_data = df_ticket_data.drop(columns=['id_city_d'])
df_ticket_data.head()


,id,company,o_station,d_station,price_in_cents,middle_stations,other_companies,o_city,d_city,departure_datetime,arrival_datetime,duration_datetime,search_datetime,price_in_euros,o_lat,o_long,d_lat,d_long
0,6795025,8385,NaN,NaN,4550,NaN,NaN,611,542,2017-10-13 14:00:00,2017-10-13 20:10:00,0 days 06:10:00,2017-10-01 00:13:31,45.5,47.907018,1.90627,43.604452,3.918318
1,6795026,9,63.0,1044.0,1450,"{149,418}",{13},611,542,2017-10-13 13:05:00,2017-10-14 06:55:00,0 days 17:50:00,2017-10-01 00:13:35,14.5,47.907018,1.90627,43.604452,3.918318
2,6795027,8377,5905.0,6495.0,7400,"{798,798,6794,6246}","{8377,8376}",611,542,2017-10-13 13:27:00,2017-10-14 21:24:00,1 days 07:57:00,2017-10-01 00:13:40,74.0,47.907018,1.90627,43.604452,3.918318
3,6795028,8377,5905.0,6495.0,13500,"{798,798,6794,6246}","{8377,8376}",611,542,2017-10-13 13:27:00,2017-10-14 11:02:00,0 days 21:35:00,2017-10-01 00:13:40,135.0,47.907018,1.90627,43.604452,3.918318
4,6795029,8381,5905.0,6495.0,7710,"{5983,5983}",{8380},611,542,2017-10-13 21:46:00,2017-10-14 19:32:00,0 days 21:46:00,2017-10-01 00:13:40,77.1,47.907018,1.90627,43.604452,3.918318


Given the information we have, we will compute the distance as the crow flies using the latitude and longitude of cities and stations respectively given by the datasets `cities.csv` and `stations.csv`. 
<br>

In [16]:
r_earth=6378 # km
def compute_distance(o_lat, o_long, d_lat, d_long):
    """
    Parameters: 
        - o_lat: float
            Latitude of the origin.
        - o_long: float
            Longitude of the origin.
        - d_lat: float
            Latitude of the destination.
        - d_long: float
            Longitude of the destination.
    Returns:
        - dist: float
            Distance (as the crow flies) between the origin and the destination expressed in km
    """
    delta_lat = abs(d_lat-o_lat)
    delta_long = abs(d_long-o_long)
    return r_earth/90*np.sqrt(delta_lat*delta_lat + delta_long*delta_long) #in km

In [17]:
# As before, we will use the function `apply()` to facilitate the tasks.
df_ticket_data['distance'] = df_ticket_data.apply(lambda x: compute_distance(x.o_lat, x.o_long, x.d_lat, x.d_long), axis=1)

In [18]:
df_ticket_data

,id,company,o_station,d_station,price_in_cents,middle_stations,other_companies,o_city,d_city,departure_datetime,arrival_datetime,duration_datetime,search_datetime,price_in_euros,o_lat,o_long,d_lat,d_long,distance
0,6795025,8385,NaN,NaN,4550,NaN,NaN,611,542,2017-10-13 14:00:00,2017-10-13 20:10:00,0 days 06:10:00,2017-10-01 00:13:31,45.5,47.907018,1.906270,43.604452,3.918318,336.601086
1,6795026,9,63.0,1044.0,1450,"{149,418}",{13},611,542,2017-10-13 13:05:00,2017-10-14 06:55:00,0 days 17:50:00,2017-10-01 00:13:35,14.5,47.907018,1.906270,43.604452,3.918318,336.601086
2,6795027,8377,5905.0,6495.0,7400,"{798,798,6794,6246}","{8377,8376}",611,542,2017-10-13 13:27:00,2017-10-14 21:24:00,1 days 07:57:00,2017-10-01 00:13:40,74.0,47.907018,1.906270,43.604452,3.918318,336.601086
3,6795028,8377,5905.0,6495.0,13500,"{798,798,6794,6246}","{8377,8376}",611,542,2017-10-13 13:27:00,2017-10-14 11:02:00,0 days 21:35:00,2017-10-01 00:13:40,135.0,47.907018,1.906270,43.604452,3.918318,336.601086
4,6795029,8381,5905.0,6495.0,7710,"{5983,5983}",{8380},611,542,2017-10-13 21:46:00,2017-10-14 19:32:00,0 days 21:46:00,2017-10-01 00:13:40,77.1,47.907018,1.906270,43.604452,3.918318,336.601086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74163,6869182,13,279.0,10729.0,2590,"{408,408}",{8371},628,562,2017-10-27 02:30:00,2017-10-27 16:00:00,0 days 13:30:00,2017-10-19 10:35:42,25.9,48.847702,2.352311,47.248785,-1.520898,296.949789
74164,6869185,9,279.0,304.0,2700,"{1105,1105}",{9},628,562,2017-10-27 07:00:00,2017-10-27 13:30:00,0 days 06:30:00,2017-10-19 10:35:42,27.0,48.847702,2.352311,47.248785,-1.520898,296.949789
74165,6869187,8371,10642.0,304.0,3000,"{1105,1105}",{9},628,562,2017-10-27 08:30:00,2017-10-27 15:15:00,0 days 06:45:00,2017-10-19 10:35:42,30.0,48.847702,2.352311,47.248785,-1.520898,296.949789
74166,6869189,13,279.0,304.0,2880,"{863,863}",{13},628,562,2017-10-27 13:25:00,2017-10-27 21:10:00,0 days 07:45:00,2017-10-19 10:35:42,28.8,48.847702,2.352311,47.248785,-1.520898,296.949789


<div class="alert alert-info">
    
A more accurate/relevant computation for the distance would take into account the different stations. 

$$d_{total} = \sum_{i=0}^{n-1} d_{i,i+1}$$ where $d_{i,i+1}$ denotes the distance as crow flies between station at time $i$ and $i+1$
</div>
I opted for this solution because it was simpler and also because there were NaN values in columns corresponding to `o_station` and `d_station`.

Let us now add a column called labeled_distance which enables to segment thhe distance in 4 parts:
* 0-200km
* 200-800km
* 800-2000km
* 2000+km

In [19]:
def segment_distance(dist):
    if dist <=200:
        return "0_200km"
    if dist <= 800:
        return "200_800km"
    if dist <=2000:
        return "800_2000km"
    if dist <= 800:
        return "2000+km"

df_ticket_data["labeled_distance"] = df_ticket_data["distance"].apply(lambda x: segment_distance(x))
df_ticket_data

,id,company,o_station,d_station,price_in_cents,middle_stations,other_companies,o_city,d_city,departure_datetime,arrival_datetime,duration_datetime,search_datetime,price_in_euros,o_lat,o_long,d_lat,d_long,distance,labeled_distance
0,6795025,8385,NaN,NaN,4550,NaN,NaN,611,542,2017-10-13 14:00:00,2017-10-13 20:10:00,0 days 06:10:00,2017-10-01 00:13:31,45.5,47.907018,1.906270,43.604452,3.918318,336.601086,200_800km
1,6795026,9,63.0,1044.0,1450,"{149,418}",{13},611,542,2017-10-13 13:05:00,2017-10-14 06:55:00,0 days 17:50:00,2017-10-01 00:13:35,14.5,47.907018,1.906270,43.604452,3.918318,336.601086,200_800km
2,6795027,8377,5905.0,6495.0,7400,"{798,798,6794,6246}","{8377,8376}",611,542,2017-10-13 13:27:00,2017-10-14 21:24:00,1 days 07:57:00,2017-10-01 00:13:40,74.0,47.907018,1.906270,43.604452,3.918318,336.601086,200_800km
3,6795028,8377,5905.0,6495.0,13500,"{798,798,6794,6246}","{8377,8376}",611,542,2017-10-13 13:27:00,2017-10-14 11:02:00,0 days 21:35:00,2017-10-01 00:13:40,135.0,47.907018,1.906270,43.604452,3.918318,336.601086,200_800km
4,6795029,8381,5905.0,6495.0,7710,"{5983,5983}",{8380},611,542,2017-10-13 21:46:00,2017-10-14 19:32:00,0 days 21:46:00,2017-10-01 00:13:40,77.1,47.907018,1.906270,43.604452,3.918318,336.601086,200_800km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74163,6869182,13,279.0,10729.0,2590,"{408,408}",{8371},628,562,2017-10-27 02:30:00,2017-10-27 16:00:00,0 days 13:30:00,2017-10-19 10:35:42,25.9,48.847702,2.352311,47.248785,-1.520898,296.949789,200_800km
74164,6869185,9,279.0,304.0,2700,"{1105,1105}",{9},628,562,2017-10-27 07:00:00,2017-10-27 13:30:00,0 days 06:30:00,2017-10-19 10:35:42,27.0,48.847702,2.352311,47.248785,-1.520898,296.949789,200_800km
74165,6869187,8371,10642.0,304.0,3000,"{1105,1105}",{9},628,562,2017-10-27 08:30:00,2017-10-27 15:15:00,0 days 06:45:00,2017-10-19 10:35:42,30.0,48.847702,2.352311,47.248785,-1.520898,296.949789,200_800km
74166,6869189,13,279.0,304.0,2880,"{863,863}",{13},628,562,2017-10-27 13:25:00,2017-10-27 21:10:00,0 days 07:45:00,2017-10-19 10:35:42,28.8,48.847702,2.352311,47.248785,-1.520898,296.949789,200_800km


### Transport Type
Now we want to get the information about the transport type for all the records in `df_tickets_data`. 
<br>The transport type is in the dataframe `df_providers`. We need to find the key to correctly join this dataframe from `df_tickets_data`. 
<br>There are not a lot of options. An easy way is to use thanks to the attribtes related to the companies.
<br>Unfortunately, as we can see below, there are 2 companies id associated to multiple transport types in the dataframe `df_providers`. 

In [20]:
# The problem concerns the company_id 9 and 20.
df_providers[["company_id", "transport_type"]].groupby(["company_id"]).count()

,transport_type
company_id,
1,1
2,1
3,1
4,1
5,1
6,1
8,1
9,201
10,11


In [21]:
df_ticket_data[df_ticket_data["company"]==20]
# Fortunately, there are no records with company 20

,id,company,o_station,d_station,price_in_cents,middle_stations,other_companies,o_city,d_city,departure_datetime,arrival_datetime,duration_datetime,search_datetime,price_in_euros,o_lat,o_long,d_lat,d_long,distance,labeled_distance


In [31]:
df_providers.transport_type.unique()

array(['bus', 'car', 'train', 'carpooling'], dtype=object)

In [23]:
# But there are 3560 records with company 9. 
df_ticket_data[df_ticket_data["company"]==9]
print("3560/74168*100 =", 3560/74168*100)
# Still, 3560 represents less than 5% of the all the records. Thus, an easy solution is to not take it ito account.

,id,company,o_station,d_station,price_in_cents,middle_stations,other_companies,o_city,d_city,departure_datetime,arrival_datetime,duration_datetime,search_datetime,price_in_euros,o_lat,o_long,d_lat,d_long,distance,labeled_distance
1,6795026,9,63.0,1044.0,1450,"{149,418}",{13},611,542,2017-10-13 13:05:00,2017-10-14 06:55:00,0 days 17:50:00,2017-10-01 00:13:35,14.5,47.907018,1.906270,43.604452,3.918318,336.601086,200_800km
51,6795076,9,10144.0,396.0,1990,"{932,932}",{13},628,453,2017-10-06 10:05:00,2017-10-06 16:40:00,0 days 06:35:00,2017-10-01 01:04:12,19.9,48.847702,2.352311,50.638756,3.076675,136.913559,0_200km
53,6795078,9,10144.0,396.0,1800,"{932,932}",{13},628,453,2017-10-06 11:55:00,2017-10-07 18:45:00,1 days 06:50:00,2017-10-01 01:04:12,18.0,48.847702,2.352311,50.638756,3.076675,136.913559,0_200km
55,6795079,9,3.0,396.0,1590,"{406,406}",{13},628,453,2017-10-06 12:30:00,2017-10-06 16:40:00,0 days 04:10:00,2017-10-01 01:04:12,15.9,48.847702,2.352311,50.638756,3.076675,136.913559,0_200km
57,6795081,9,10144.0,396.0,1100,"{406,406}",{13},628,453,2017-10-06 17:40:00,2017-10-07 18:45:00,1 days 01:05:00,2017-10-01 01:04:12,11.0,48.847702,2.352311,50.638756,3.076675,136.913559,0_200km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74047,6869107,9,279.0,860.0,2700,"{63,63}",{9},628,126,2017-10-20 12:15:00,2017-10-20 23:35:00,0 days 11:20:00,2017-10-19 10:30:37,27.0,48.847702,2.352311,44.833570,-0.573845,352.027225,200_800km
74053,6869106,9,279.0,860.0,2990,"{500,245}",{13},628,126,2017-10-20 06:30:00,2017-10-20 18:55:00,0 days 12:25:00,2017-10-19 10:30:37,29.9,48.847702,2.352311,44.833570,-0.573845,352.027225,200_800km
74054,6869108,9,279.0,860.0,3790,"{500,500}",{13},628,126,2017-10-20 16:30:00,2017-10-21 03:00:00,0 days 10:30:00,2017-10-19 10:30:37,37.9,48.847702,2.352311,44.833570,-0.573845,352.027225,200_800km
74158,6869184,9,279.0,10729.0,2400,"{1105,1105}",{8371},628,562,2017-10-27 07:00:00,2017-10-27 16:00:00,0 days 09:00:00,2017-10-19 10:35:42,24.0,48.847702,2.352311,47.248785,-1.520898,296.949789,200_800km


In [24]:
df_tickets_data_simple = df_ticket_data[df_ticket_data["company"]!=9]

In [25]:
# Beware: execute the cell just once
df_tickets_data_simple = pd.merge(left=df_tickets_data_simple, right=df_providers[['company_id', 'transport_type']], how='left', left_on='company', right_on='company_id', suffixes=("", ""))
df_tickets_data_simple = df_tickets_data_simple.drop(columns=['company_id'])

In [26]:
df_tickets_data_simple.head()

,id,company,o_station,d_station,price_in_cents,middle_stations,other_companies,o_city,d_city,departure_datetime,...,duration_datetime,search_datetime,price_in_euros,o_lat,o_long,d_lat,d_long,distance,labeled_distance,transport_type
0,6795025,8385,NaN,NaN,4550,NaN,NaN,611,542,2017-10-13 14:00:00,...,0 days 06:10:00,2017-10-01 00:13:31,45.5,47.907018,1.906270,43.604452,3.918318,336.601086,200_800km,NaN
1,6795027,8377,5905.0,6495.0,7400,"{798,798,6794,6246}","{8377,8376}",611,542,2017-10-13 13:27:00,...,1 days 07:57:00,2017-10-01 00:13:40,74.0,47.907018,1.906270,43.604452,3.918318,336.601086,200_800km,NaN
2,6795028,8377,5905.0,6495.0,13500,"{798,798,6794,6246}","{8377,8376}",611,542,2017-10-13 13:27:00,...,0 days 21:35:00,2017-10-01 00:13:40,135.0,47.907018,1.906270,43.604452,3.918318,336.601086,200_800km,NaN
3,6795029,8381,5905.0,6495.0,7710,"{5983,5983}",{8380},611,542,2017-10-13 21:46:00,...,0 days 21:46:00,2017-10-01 00:13:40,77.1,47.907018,1.906270,43.604452,3.918318,336.601086,200_800km,NaN
4,6795030,8385,NaN,NaN,1800,NaN,NaN,628,453,2017-10-06 05:30:00,...,0 days 03:00:00,2017-10-01 01:03:18,18.0,48.847702,2.352311,50.638756,3.076675,136.913559,0_200km,NaN


In [33]:
df_tickets_data_simple.groupby(["labeled_distance"]).count()
# There are no tickets with a travel distance exceeding 2000km.

,id,company,o_station,d_station,price_in_cents,middle_stations,other_companies,o_city,d_city,departure_datetime,arrival_datetime,duration_datetime,search_datetime,price_in_euros,o_lat,o_long,d_lat,d_long,distance,transport_type
labeled_distance,,,,,,,,,,,,,,,,,,,,
0_200km,25954,25954,6404,6404,25954,6404,6404,25954,25954,25954,25954,25954,25954,25954,25954,25954,25954,25954,25954,1
200_800km,44312,44312,22456,22456,44312,22456,22456,44312,44312,44312,44312,44312,44312,44312,44312,44312,44312,44312,44312,103
800_2000km,432,432,397,397,432,397,397,432,432,432,432,432,432,432,432,432,432,432,432,0


In [35]:
def my_agg_mean(x):
    names = {
        'Number': x['price_in_euros'].count(),
        'Price mean': x['price_in_euros'].mean(),
        'Duration mean': x['duration_datetime'].mean(),
    }

    return pd.Series(names, index=[ 'Number', 
                                    'Price mean', 
                                   'Duration mean'])


df_tickets_data_simple.groupby(["labeled_distance", "transport_type"]).apply(my_agg_mean)

Number  Price mean   Duration mean
labeled_distance transport_type                                    
0_200km          carpooling           1   19.900000        14:15:00
200_800km        bus                  9   32.655556 12:42:13.333333
                 carpooling           4   43.385000        10:36:15
                 train               90   32.655556 12:42:13.333333

We observe that (if we do not take into account the records with company 9) there are no either no transport type valid for distance travel superior to 800km.
<br>The join on the id of the company cause important loss.